In [27]:
import os
import cv2
import numpy as np

# Function to load and preprocess images
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if img_path.endswith(".jpg") or img_path.endswith(".png"):  # Filter image files
            # Load and preprocess image
            img = cv2.imread(img_path)
            img = cv2.resize(img, (224, 224))  # Resize image to 224x224
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            # Append image and label to lists
            images.append(img)
            labels.append(label)
    return images, labels

# Paths to folders containing images
normal_folder = "artifacts/data_ingestion/kidney-ct-scan-image/Normal"
tumor_folder = "artifacts/data_ingestion/kidney-ct-scan-image/Tumor"

# Load normal images and assign label 0
normal_images, normal_labels = load_images_from_folder(normal_folder, 0)

# Load tumor images and assign label 1
tumor_images, tumor_labels = load_images_from_folder(tumor_folder, 1)

# Combine normal and tumor images and labels
X_test = np.array(normal_images + tumor_images)
y_test = np.array(normal_labels + tumor_labels)

# Convert labels to one-hot encoding (optional)
# y_test = np.eye(2)[y_test]

# Shuffle X_test and y_test in the same order
shuffle_indices = np.random.permutation(len(X_test))
X_test = X_test[shuffle_indices]
y_test = y_test[shuffle_indices]

# Normalize pixel values to range [0, 1]
X_test = X_test.astype('float32') / 255.0

# Print shapes of X_test and y_test
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_test shape: (7360, 224, 224, 3)
y_test shape: (7360,)


In [29]:
type(X_test)

numpy.ndarray

In [30]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the model
model = load_model('artifacts/training/model.h5')

# Load the dataset
X_test = X_test  # Your test dataset features
y_test = y_test 

In [31]:
y_pred = model.predict(X_test)

230/230 [==============================] - 703s 3s/step


In [32]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, r2_score

In [44]:
accuracy = accuracy_score(y_test, y_pred.argmax(axis=1))
print("Accuracy:", accuracy)

Accuracy: 0.9455163043478261


In [34]:
conf_matrix = confusion_matrix(y_test, y_pred.argmax(axis=1))
print("Confusion Matrix:\n", conf_matrix)

Confusion Matrix:
 [[4966  111]
 [ 290 1993]]


In [35]:
precision = precision_score(y_test, y_pred.argmax(axis=1))
print("Precision:", precision)

Precision: 0.9472433460076045


In [36]:
recall = recall_score(y_test, y_pred.argmax(axis=1))
print("Recall:", recall)

Recall: 0.8729741568112133
